In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import os
import matplotlib
data_path = "gogs_test_data_real/raft_stereo1/depth"

In [ ]:
def load_zed_depth_folder(pattern):
    files = glob.glob(pattern)
    depth_ims = np.zeros((len(files), 720, 1280, 1))
    file_names = []
    for i, file in enumerate(files):
        float_values = np.loadtxt(file)
        file_names.append(file)

        np_arr = np.array(float_values).reshape(720, 1280, 1)
        depth_ims[i] = np_arr
    return depth_ims, file_names

pattern = os.path.join(data_path, "*.txt")
depth_ims, file_names = load_zed_depth_folder(pattern)
print(np.isinf(depth_ims).sum())
print(np.isnan(depth_ims).sum())

In [ ]:
print(depth_ims.shape)
print(np.isnan(depth_ims).sum())
print(np.isinf(depth_ims).sum())

def plot_arr(depth_ims, file_names=None, max_imgs=None):
    cols = 4
    max_imgs = min(max_imgs, len(depth_ims)) if max_imgs is not None else len(depth_ims)
    # heights = {'height_ratios': [780]*((max_imgs // cols + 1)*cols)}
    fig, axes = plt.subplots(max_imgs // cols + 1, cols, figsize=(12, 16))
    for i, img in enumerate(depth_ims[:max_imgs]):
        axes[i//cols, i%cols].imshow(img)
        if file_names is not None:
            axes[i//4, i%4].set_title(f'File: {Path(file_names[i]).stem}')
        
    for ax in axes.flatten():
        ax.axis('off')

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.tight_layout()
    plt.show()
plot_arr(depth_ims, file_names, max_imgs=28)

In [ ]:
def img_nan_clean(input_img):
    # new_size = (512, 288)
    # new_size = (720, 1280)
    # image = cv2.resize(input_img.copy(), new_size)
    # image[np.isnan(image)] = np.nanmin(image)
    # image = image.astype(np.float32)

    # mask = np.isnan(cv2.resize(input_img, new_size)).astype(np.uint8)

    # cleaned_array = cv2.inpaint(image, mask, 5, cv2.INPAINT_TELEA)
    # cleaned_array = cleaned_array.clip(0, 1000)

    # assert np.isnan(cleaned_array).sum() == 0, "still NaNs in image"
    cleaned_array = input_img.copy()
    # cleaned_array[np.isnan(cleaned_array)] = np.percentile(input_img[~np.isnan(cleaned_array)], 10)

    ### HARDCODED TABLE HEIGHT
    cleaned_array[np.isnan(cleaned_array) | (cleaned_array > 1)] = .4

    return cleaned_array

cleaned_depth_ims = np.array([img_nan_clean(depth_im) for depth_im in depth_ims])
# plot_arr(cleaned_depth_ims[:,:,100:750], file_names = file_names, max_imgs = 28)
plot_arr(cleaned_depth_ims, file_names = file_names, max_imgs = 28)


In [ ]:
import torch
from dexnet.grasp_model import HighResFCGQCNN as FCGQCNN
w = torch.load("/home/apgoldberg/Dex-Net-5.0/model_zoo/max_normal_normal_conversion.pth")
# from dexnet.grasp_model import DexNet3FCGQCNN as FCGQCNN
# w = torch.load("/home/apgoldberg/Dex-Net-5.0/model_zoo/ryans_fcgqcnn_conversion.pt")
model = FCGQCNN()
model.load_state_dict(w)
model.to("cuda")
model.eval()

def preprocess_img(x, normal_crop=None):
    # x = 255 - x # blender depth is inverted

    kernel = np.ones((5,5), np.uint8)
    x = cv2.dilate(x, kernel, iterations=1)
    x = cv2.GaussianBlur(x, (5, 5), 5)
    if normal_crop is None:
        x = (x - x.mean()) / x.std()
    elif normal_crop == "normal":
        normalizers = (
            0.59784445,
            # 0.00770147890625,
            .06873399
        )  # mean, std value for depth images (from analyze.py script)
        x = (x - normalizers[0]) / normalizers[1]
    else:
        x_crop = x[normal_crop[0]:normal_crop[1],normal_crop[2]:normal_crop[3]]
        x = (x - x_crop.mean()) / x_crop.std()
    # x *= 10
    # x = cv2.resize(x, (60,60)) # 125,70
    # pad = 15
    # x = cv2.copyMakeBorder(x, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    x = x.squeeze()
    x = torch.tensor(x, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to("cuda")
    return x
    verlay


In [ ]:
print(file_names)
print(cleaned_depth_ims.shape)
# rgb_img_path = os.path.join(Path(file_names[3]).parent.parent, "img", f"{Path(file_names[3]).stem}_l.jpg")
rgb_img_path = "/home/apgoldberg/Dex-Net-5.0/gogs_test_data_real/raft_stereo1/img/0_l.jpg"
rgb_img = plt.imread(rgb_img_path)
rgb_img = rgb_img[:,:]
print(rgb_img.shape)
plt.imshow(rgb_img)

In [ ]:
def composite_rgb_grasp(rgb_img, grasp_out):
    cmap = matplotlib.colormaps['rainbow']
    target_height, target_width= rgb_img.shape[0], rgb_img.shape[1]

    grasp_out_resized = (cv2.resize(
        (grasp_out.squeeze()),
        (target_width, target_height)
    ))

    color_mapped_grasps = cmap(grasp_out_resized)[:,:,:3]
    to_show_alpha = (grasp_out_resized.squeeze() < .2)[:,:,None]

    to_show = (cv2.resize(color_mapped_grasps, (target_width, target_height))).astype(np.uint8)
    to_show = .5*to_show_alpha*to_show + (rgb_img).astype(np.uint8)

    to_show = to_show.astype(np.uint8)
    return to_show

def plot_squares_with_numbers(arr):
    viridis = matplotlib.colormaps['viridis']
    color_div, color_add = np.nanmax(arr) - np.nanmin(arr), np.nanmin(arr)
    dim1, dim2 = arr.shape[0], arr.shape[1]
    fig, ax = plt.subplots(figsize=(32, 32))
    for i in range(dim1):
        for j in range(dim2):
            color = viridis((arr[i][j] - color_add) / color_div)
            ax.add_patch(plt.Rectangle((j, dim1 - i - 1), 1, 1, color=color))
            ax.text(j + 0.5, dim1 - i - 1 + 0.5, str(round(arr[i][j], 2)), ha='center', va='center')

    ax.set_xlim(0, dim2)
    ax.set_ylim(0, dim1)
    ax.axis('off')
    plt.show()

In [ ]:
base_input_im = cleaned_depth_ims[3] + .35
# base_input_im = cleaned_depth_ims[3,:-50,200:-50] + .3
# base_input_im = cleaned_depth_ims[3,200:500,1000:1250] + .3
# base_input_im[base_input_im > .775] = .84
# base_input_im = cv2.resize(base_input_im, (34,34))
plt.imshow(base_input_im)
plt.show()

def inference(input_im):
    with torch.no_grad():
        output = model(input_im)
    print(torch.max(output))
    to_show = output.cpu().squeeze().unsqueeze(-1).numpy()
    # plt.imshow(output.squeeze().cpu(), vmin=0, vmax=1)
    # plt.show()
    return output, to_show

scales = np.arange(.05,.4,.05)
# scales = [1]
fig, axes = plt.subplots(max(len(scales) // 4 + 1, 2), 4, figsize=(16, 12))
for i, scale in enumerate(scales):
    # Resize the input image
    resized_img = cv2.resize(base_input_im, None, fx=scale, fy=scale)
    
    # Preprocess the resized image
    input_im = preprocess_img(resized_img, normal_crop="normal")
    # plt.imshow(input_im.squeeze().cpu())
    # plt.show()
 
    # Pass the preprocessed image through the model
    output, to_show = inference(input_im)
    plt.show(to_show)
    # pad = 15
    # to_show = cv2.copyMakeBorder(to_show, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=0)
    # # plt.imshow(to_show, vmin=0, vmax=1)
    # # plt.show()
    
    # # Prepare the output for visualization
    # # to_show_rgb = cv2.resize(rgb_img, None, fx=scale, fy=scale)
    # cmap = matplotlib.colormaps['gray']
    # to_show_rgb = cmap(resized_img.squeeze())[:,:,:3]
    # to_show[to_show < .2] = 0
    # # to_show = composite_rgb_grasp(to_show_rgb, to_show)

    axes[i//4, i%4].imshow(to_show)
    axes[i//4, i%4].set_title(f'Scale: {scale}')
    axes[i//4, i%4].axis('off')

plt.tight_layout()
plt.show()
# plot_squares_with_numbers(base_input_im.squeeze())

In [ ]:
scale = .3
z_offsets = np.arange(.1, .2, .01)

fig, axes = plt.subplots(max(len(z_offsets) // 4 + 1, 2), 4, figsize=(16, 12))
for i, z_offset in enumerate(z_offsets):
    base_input_im = cleaned_depth_ims[3] + z_offset
    resized_img = cv2.resize(base_input_im, None, fx=scale, fy=scale)
    input_im = preprocess_img(resized_img, normal_crop="normal")

    # Pass the preprocessed image through the model
    output, to_show = inference(input_im)
    pad = 15
    to_show = cv2.copyMakeBorder(to_show, pad, pad, pad, pad, cv2.BORDER_CONSTANT, value=0)
    
    # Prepare the output for visualization
    # to_show_rgb = cv2.resize(rgb_img, None, fx=scale, fy=scale)
    cmap = matplotlib.colormaps['gray']
    to_show_rgb = cmap(resized_img.squeeze())[:,:,:3]
    to_show[to_show < .2] = 0
    to_show = composite_rgb_grasp(to_show_rgb, to_show)

    axes[i//4, i%4].imshow(to_show)
    axes[i//4, i%4].set_title(f'z_add: {z_offset}')
    axes[i//4, i%4].axis('off')

plt.tight_layout()
plt.show()

In [ ]:

plt.imshow(input_im.cpu().squeeze().unsqueeze(-1).numpy(), cmap="gray")